In [1]:
from Preprocessor import Preprocessor

preprocessor = Preprocessor()

Preprocessor is Operating


In [2]:
import pandas as pd
from sklearn.metrics import log_loss

train = preprocessor.preprocessing('../data/train.csv')
print(train.shape)
train.head(3)

(26457, 25)


,gender,car,reality,income_total,edu_type,family_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,...,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working
0,0,0,0,202500.0,0,0,-13899,-4709,0,0,...,0,1,0,0,0,1,0,0,0,0
1,0,0,1,247500.0,1,1,-11380,-1540,0,0,...,1,0,0,0,0,1,0,0,0,0
2,1,1,1,450000.0,0,0,-19087,-4434,0,1,...,1,0,0,0,0,0,0,0,0,1


In [3]:
test = preprocessor.preprocessing('../data/test.csv')
print(test.shape)
test.head(3)

(10000, 24)


,gender,car,reality,income_total,edu_type,family_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,...,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working
0,1,1,0,112500.0,1,1,-21990,0,0,1,...,1,0,0,0,0,0,1,0,0,0
1,0,0,1,135000.0,0,0,-18964,-8671,0,1,...,1,0,0,0,0,0,0,1,0,0
2,0,0,1,69372.0,1,0,-15887,-217,1,1,...,1,0,0,0,0,0,0,0,0,1


In [4]:
test.isnull().sum()

gender                              0
car                                 0
reality                             0
income_total                        0
edu_type                            0
family_type                         0
DAYS_BIRTH                          0
DAYS_EMPLOYED                       0
work_phone                          0
phone                               0
email                               0
family_size                         0
begin_month                         0
house_type_Co-op apartment          0
house_type_House / apartment        0
house_type_Municipal apartment      0
house_type_Office apartment         0
house_type_Rented apartment         0
house_type_With parents             0
income_type_Commercial associate    0
income_type_Pensioner               0
income_type_State servant           0
income_type_Student                 0
income_type_Working                 0
dtype: int64

In [5]:
from pycaret.classification import *

clf = setup(data=train, target='credit', train_size=0.8)

,Description,Value
0,session_id,3986
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 25)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function               <function log_loss at 0x000002547FC5DAE8>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [7]:
best2 = compare_models(fold=5, sort='logloss', n_select=2, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6989,0.7087,0.4298,0.6948,0.6242,0.2442,0.3252,0.7638,0.2980
gbc,Gradient Boosting Classifier,0.6908,0.6580,0.4103,0.6574,0.6060,0.2101,0.2991,0.7961,1.9820
lda,Linear Discriminant Analysis,0.6422,0.6091,0.3369,0.5421,0.5069,0.0107,0.0509,0.8627,0.1540
nb,Naive Bayes,0.6490,0.6063,0.3459,0.5905,0.5211,0.0370,0.1181,0.8674,0.1260
lr,Logistic Regression,0.6406,0.5917,0.3333,0.4104,0.5003,0.0000,0.0000,0.8708,1.2700
rf,Random Forest Classifier,0.6979,0.7465,0.5303,0.6741,0.6790,0.3617,0.3696,0.9657,0.7400
ada,Ada Boost Classifier,0.6907,0.6301,0.4056,0.6487,0.6014,0.2005,0.3026,1.0807,0.3040
et,Extra Trees Classifier,0.6722,0.7083,0.5099,0.6484,0.6550,0.3140,0.3195,2.4148,0.7280
knn,K Neighbors Classifier,0.6163,0.6631,0.4676,0.5967,0.6041,0.2185,0.2204,3.9341,0.8680
dt,Decision Tree Classifier,0.6052,0.6373,0.4916,0.6156,0.6099,0.2556,0.2559,13.2428,0.1560


In [8]:
tuned_best2 = [tune_model(i, optimize='logloss') for i in best2]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.7057,0.7259,0.4717,0.6896,0.6548,0.2985,0.3490,0.7544
1,0.7109,0.7453,0.4829,0.6908,0.6640,0.3206,0.3652,0.7354
2,0.7194,0.7499,0.4910,0.7038,0.6731,0.3400,0.3876,0.7310
3,0.7133,0.7498,0.4764,0.6921,0.6624,0.3202,0.3697,0.7393
4,0.7114,0.7247,0.4750,0.6919,0.6603,0.3155,0.3643,0.7515
5,0.7027,0.7316,0.4624,0.6775,0.6489,0.2905,0.3393,0.7485
6,0.7046,0.7414,0.4763,0.6815,0.6565,0.3039,0.3486,0.7511
7,0.7051,0.7297,0.4715,0.6799,0.6555,0.3041,0.3478,0.7520
8,0.7023,0.7342,0.4634,0.6785,0.6491,0.2934,0.3382,0.7571
9,0.7051,0.7375,0.4712,0.6799,0.6549,0.3048,0.3486,0.7581


In [9]:
blend_best2 = blend_models(estimator_list=tuned_best2, fold=5, optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.7127,0.7317,0.4780,0.6999,0.6620,0.3143,0.3684,0.7463
1,0.7118,0.7387,0.4757,0.6895,0.6613,0.3168,0.3658,0.7394
2,0.7142,0.7322,0.4785,0.7030,0.6631,0.3197,0.3713,0.7437
3,0.7066,0.7325,0.4778,0.6875,0.6586,0.3085,0.3526,0.7480
4,0.7064,0.7299,0.4710,0.6844,0.6553,0.3062,0.3510,0.7562
Mean,0.7103,0.7330,0.4762,0.6928,0.6601,0.3131,0.3618,0.7467
SD,0.0032,0.0030,0.0028,0.0072,0.0028,0.0051,0.0084,0.0056


In [10]:
pred = predict_model(blend_best2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.7111,0.7448,0.4771,0.6937,0.6621,0.3126,0.3575,0.7304


In [11]:
final_model = finalize_model(blend_best2)
prep_pipe = get_config('prep_pipe')
prep_pipe.steps.append(['trained_model', final_model])
pred = prep_pipe.predict_proba(test)

# final_model = finalize_model(blend_best2)
# pred = predict_model(final_model, data=test)

submission = pd.read_csv('../data/sample_submission.csv')
submission.iloc[:, 1:] = pred
submission.head()

,index,0,1,2
0,26457,0.041256,0.108067,0.850677
1,26458,0.180269,0.178514,0.641216
2,26459,0.037054,0.079341,0.883605
3,26460,0.121383,0.126483,0.752134
4,26461,0.096786,0.207435,0.695779


In [12]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_pycaret.csv', index =False)

오늘 날짜 : 20210520
